In [1]:
import pandas as pd
import numpy as np
from anndata import AnnData
import anndata as ad
import scanpy as sc
import scipy.sparse as sp
import glob
from tqdm import tqdm
from paste import pairwise_align
import ot
import torch
import matplotlib.pyplot as plt

In [2]:
l = torch.nn.Linear(10,10).to('cuda:1')

In [3]:
all_index = [25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 147, 149, 151, 153, 155]

In [4]:
h5ad_dic = '/home/hanyuji/Data/ST_data/macaque_cortex/h5ad/'
files = 'total_gene_T25.type20230503-macaque1-contour2.h5ad'

data_index = all_index

adatas = []
for index in tqdm(data_index):
    data_path = h5ad_dic + f'total_gene_T{index}.type20230503-macaque1-contour2.h5ad'
    adata = sc.read_h5ad(data_path)
    
    # 简单的预处理
    adata.obs['slice_index'] = np.repeat(index, adata.n_obs)  # 添加切片序号信息
    adata.obsm['spatial_raw'] = adata.obsm['spatial'].copy()  # 复制obsm['spatial']中的数据
    adata.obsm['spatial'] = np.delete(adata.obsm['spatial'], [0, 1], axis=1)  # 删除obsm['spatial']中的前两列
    
    adatas.append(adata)

100%|██████████| 119/119 [09:05<00:00,  4.59s/it]


In [5]:
# 合并多个AnnData对象
combined_adata = ad.concat(
    adatas, 
    axis=0, 
    join='outer', 
    merge='unique', 
    uns_merge='unique', 
    label='batch', 
    keys=list(range(len(adatas))), 
    index_unique='-', 
    fill_value=0
)


In [6]:
combined_adata

AnnData object with n_obs × n_vars = 30975704 × 15929
    obs: 'chip', 'gene_area', 'slice_index', 'batch'
    obsm: 'spatial', 'spatial_raw'

In [7]:
file_path = '/home/hanyuji/Workbench/ST/ST_data_check/paste_alignment_cortex/PASTE_align/cortex_h5ad_2000_HVG/cortex_macaque1_119slice_30975704spot_15929gene_combined.h5ad'
combined_adata.write(file_path)

In [10]:
def get_slice_HVG(slice, n_top_genes=2000, need_norm_and_log1p=False, target_sum=1e4):
    '''
    计算并选择前2000个高可变基因
    '''
    
    slice.var_names_make_unique()  # 确保变量名唯一，避免重复
    sc.pp.filter_genes(slice, min_counts=5)  # 过滤掉计数小于5的基因
    sc.pp.highly_variable_genes(slice, n_top_genes=n_top_genes, flavor='seurat')  # 使用seurat_v3方法计算高可变基因，并选择前3000个
    slice = slice[:, slice.var.highly_variable]  # 只保留高可变基因
    
    if need_norm_and_log1p:
        sc.pp.normalize_total(slice, target_sum=target_sum)  # 将每个细胞的总表达量归一化为10000
        sc.pp.log1p(slice)  # 对数据进行对数变换
    
    return slice  # 返回处理后的数据


# slice_0_subset_HVG = get_slice_HVG(slice_0_subset)
# slice_1_subset_HVG = get_slice_HVG(slice_1_subset)

adatas_HVG = get_slice_HVG(combined_adata)



: 

In [ ]:
adatas_HVG.obs

In [9]:
adatas_HVG

AnnData object with n_obs × n_vars = 30975704 × 15929
    obs: 'chip', 'gene_area', 'slice_index', 'batch'
    obsm: 'spatial', 'spatial_raw'

In [ ]:
file_path = '/home/hanyuji/Workbench/ST/ST_data_check/paste_alignment_cortex/PASTE_align/cortex_h5ad_2000_HVG/cortex_macaque1_119slice_2000gene_HVG.h5ad'
adatas_HVG.write(file_path)